In [5]:
from tqdm.notebook import tqdm
from api.config.db_config import get_db_connection
import random
import json
import re

from ollama import Client

client = Client(host="http://127.0.0.1:11434")
with open("sdg_label_prompt.md", "r") as f:
    prompt_template = f.read()
        


In [6]:
all_items = []
try:
    with open("../src/ai/testsets/testset_v1_en_labeled.jsonl", "r", encoding="utf-8") as f:
        for line in f:
            try:
                all_items.append(json.loads(line.strip()))
            except json.JSONDecodeError as e:
                print(f"Attention : Ligne ignorée car JSON invalide : {line.strip()} - Erreur : {e}")
except FileNotFoundError:
    print(f"Erreur : Le fichier {"../src/ai/testsets/testset_v1_en_labeled.jsonl"} n'a pas été trouvé.")
    all_items = [] # S'assurer que all_items est une liste vide si le fichier n'est pas trouvé

if all_items:
    # Séparer les items
    sdg_none_items = [item for item in all_items if item.get("sdg") == "None" or item.get("sdg") is None]
    sdg_present_items = [item for item in all_items if item.get("sdg") != "None" and item.get("sdg") is not None]

    # Calculer combien d'items "None" conserver (10%)
    num_none_to_keep = int(len(sdg_none_items) * 0.10)

    # Sélectionner aléatoirement les items "None" à conserver
    # random.sample s'assure qu'il n'y a pas de doublons
    sdg_none_items_kept = random.sample(sdg_none_items, num_none_to_keep)

    # Combiner les listes
    filtered_data = sdg_present_items + sdg_none_items_kept

    # Mélanger les résultats si désiré (optionnel)
    random.shuffle(filtered_data)

    print(f"Nombre total d'items initial : {len(all_items)}")
    print(f"Nombre d'items avec SDG == 'None' : {len(sdg_none_items)}")
    print(f"Nombre d'items avec SDG != 'None' : {len(sdg_present_items)}")
    print(f"Nombre d'items avec SDG == 'None' à conserver (10%) : {len(sdg_none_items_kept)}")
    print(f"Nombre total d'items après filtrage : {len(filtered_data)}")

    # Maintenant, 'filtered_data' contient les données souhaitées
    # Vous pouvez par exemple afficher quelques éléments pour vérifier :
    # for i, item in enumerate(filtered_data[:5]):
    # print(f"Item {i}: {item}")
else:
    print("Aucune donnée n'a été chargée du fichier.")

Nombre total d'items initial : 1727
Nombre d'items avec SDG == 'None' : 1190
Nombre d'items avec SDG != 'None' : 537
Nombre d'items avec SDG == 'None' à conserver (10%) : 119
Nombre total d'items après filtrage : 656


In [7]:
        """# Check if this description_text has already been added globally
        if description_text not in seen_description_texts_globally:

            formatted_prompt = prompt_template.replace("{description}", description_text)

            # Generate the SDG classification
            response_data = client.generate(
                model="qwen3:8b", # Make sure this model is available in your Ollama instance
                prompt=formatted_prompt
                # You might want to add other parameters like 'stream=False'
                # or options within a dictionary if your client version supports it.
                # Example: options={"temperature": 0.7}
            )
            # Get the raw text output from the model's response
            # The actual key for the response text is usually 'response'
            raw_output_text = response_data.get('response', '').strip()
            
            # 1. Try to find content within <answer>...</answer> tags (case-insensitive for tags)
            answer_match = re.search(r"<answer>(.*?)</answer>", raw_output_text, re.DOTALL | re.IGNORECASE)
            
            list_patent.append({
                "patent_number": desc_data["patent_number"], # This is actual_patent_number
                "sdg": answer_match.group(1),
                "description_number": desc_data["description_number"],
                "description_text": description_text # Store the text that was classified
            })
            
            seen_description_texts_globally.add(description_text)"""

'# Check if this description_text has already been added globally\nif description_text not in seen_description_texts_globally:\n\n    formatted_prompt = prompt_template.replace("{description}", description_text)\n\n    # Generate the SDG classification\n    response_data = client.generate(\n        model="qwen3:8b", # Make sure this model is available in your Ollama instance\n        prompt=formatted_prompt\n        # You might want to add other parameters like \'stream=False\'\n        # or options within a dictionary if your client version supports it.\n        # Example: options={"temperature": 0.7}\n    )\n    # Get the raw text output from the model\'s response\n    # The actual key for the response text is usually \'response\'\n    raw_output_text = response_data.get(\'response\', \'\').strip()\n\n    # 1. Try to find content within <answer>...</answer> tags (case-insensitive for tags)\n    answer_match = re.search(r"<answer>(.*?)</answer>", raw_output_text, re.DOTALL | re.IGNORE

In [ ]:
for data in filtered_data:
    label = data.get("sdg")
    patent_number = data.get("patent_number")
    description_number = data.get("description_number")
    description_text = data.get("description_text")
    formatted_prompt = prompt_template.replace("{description}", description_text)

    # Generate the SDG classification
    response_data = client.generate(
        model="qwen3:8b", # Make sure this model is available in your Ollama instance
        prompt=formatted_prompt
        # You might want to add other parameters like 'stream=False'
        # or options within a dictionary if your client version supports it.
        # Example: options={"temperature": 0.7}
    )
    # Get the raw text output from the model's response
    # The actual key for the response text is usually 'response'
    raw_output_text = response_data.get('response', '').strip()
            
    # 1. Try to find content within <answer>...</answer> tags (case-insensitive for tags)
    answer_match = re.search(r"<answer>(.*?)</answer>", raw_output_text, re.DOTALL | re.IGNORECASE)
    print("----------------------------------------------------------------------------------")
    
    print(raw_output_text)   
    print(f"true label :{label}")        
    

2025-05-23 23:39:12,342 - httpx - INFO - _send_single_request - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


<think>
Okay, let me try to figure out which SDG(s) this text relates to. The text is about a film material with specific properties: resistant to lidocaine, contact layer is COC, base layer is aluminum, and the tie-layer is a multilayer of a terpolymer and a copolymer. 

First, I need to understand what this material is used for. The mention of layers like COC (cyclo-olefin copolymer), aluminum, and specific polymers suggests it's a type of packaging or barrier material. Lidocaine resistance might indicate it's used in medical or pharmaceutical contexts, perhaps for drug delivery systems or protective packaging for medications. 

Looking at the SDGs, the focus here is on materials science and their applications. Let's consider each SDG:

SDG 9: Industry, Innovation, and Infrastructure. This is about sustainable industrialization, innovation, and infrastructure. Developing new materials like this could contribute to sustainable manufacturing processes, reducing waste or improving effic

2025-05-23 23:39:30,338 - httpx - INFO - _send_single_request - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


<think>
Okay, let's tackle this. The user provided a technical text about networking, specifically regarding Border Gateway Protocol (BGP) and MPLS forwarding. They want to know which SDG(s) this text relates to.

First, I need to understand the text. It's about a border node receiving a Transport-VPN route via EBGP. If the route has a PNH (probably a typo for PNH, maybe Path Node Header?) that's directly connected, like an EBGP single-hop peering address, the device checks the directly connected interface for MPLS forwarding. Then, no other nexthop resolution is done because the inter-AS link can be used for any transport class.

So the key points here are BGP, MPLS, inter-AS links, and transport classes. These are all networking concepts. Now, how do these relate to the SDGs?

Looking at the SDGs, most are about social, environmental, and economic development. But networking and infrastructure are part of the SDG 9: Industry, Innovation, and Infrastructure. That's about building resi

KeyboardInterrupt: 